# Import data and general packages

#### Install MeCab and dependencies, if necessary

#### Imports

In [2]:
import collections
import logging
import MeCab
import neologdn
import numpy as np
import os
import pandas as pd
import re
import time

from nltk.corpus import stopwords
from gensim.models.word2vec import Word2Vec

logging.basicConfig()
logging.root.setLevel(logging.INFO)

#### Get data

In [3]:
os.chdir("/home/andi/code/mochiyam/simply-japanese/data/2_RawData") # Change user name to run on your machine
os.getcwd()

'/home/andi/code/mochiyam/simply-japanese/data/2_RawData'

In [4]:
!ls

Combined_85K_10000.xlsx  SNOW_T15_10000.xlsx  SNOW_T23_10000.xlsx
Combined_85K_1000.xlsx	 SNOW_T15_1000.xlsx   SNOW_T23_1000.xlsx
Combined_85K_150.xlsx	 SNOW_T15_150.xlsx    SNOW_T23_150.xlsx
Combined_85K.xlsx	 SNOW_T15.xlsx	      SNOW_T23.xlsx


In [4]:
data = "SNOW_T15_10000.xlsx"
df = pd.read_excel(data)
data = "SNOW_T15_150.xlsx"
df150 = pd.read_excel(data)

In [6]:
df.head()

,#日本語(原文),#やさしい日本語,#英語(原文)
0,父は私が外国へ行くことを承知した。,父は私が外国へ行くことを許した。,my father consented to my going abroad .
1,卑屈な奴。,自分のことをダメだと考える人。,don 't be a dog .
2,それは本当のはずはない。,それは本当のはずはない。,it cannot be true .
3,車がそんなに混んでなければ問題ないでしょう。,車がそんなに混んでなければ問題ないでしょう。,we shouldn 't have any trouble if the traffic ...
4,２時間も待たされた。,２時間も待った。,i was kept waiting for as long as two hours .


In [5]:
X = df[df.keys()[0]]
y = df[df.keys()[1]]
X150 = df150[df150.keys()[0]]
y150 = df150[df150.keys()[1]]

In [8]:
X[0], y[0]

('父は私が外国へ行くことを承知した。', '父は私が外国へ行くことを許した。')

# Janome

In [9]:
from janome.tokenizer import Tokenizer

Simple Tokenizer

In [10]:
t = Tokenizer()
for token in t.tokenize(X[0]):
    print(token)

父	名詞,一般,*,*,*,*,父,チチ,チチ
は	助詞,係助詞,*,*,*,*,は,ハ,ワ
私	名詞,代名詞,一般,*,*,*,私,ワタシ,ワタシ
が	助詞,格助詞,一般,*,*,*,が,ガ,ガ
外国	名詞,一般,*,*,*,*,外国,ガイコク,ガイコク
へ	助詞,格助詞,一般,*,*,*,へ,ヘ,エ
行く	動詞,自立,*,*,五段・カ行促音便,基本形,行く,イク,イク
こと	名詞,非自立,一般,*,*,*,こと,コト,コト
を	助詞,格助詞,一般,*,*,*,を,ヲ,ヲ
承知	名詞,サ変接続,*,*,*,*,承知,ショウチ,ショーチ
し	動詞,自立,*,*,サ変・スル,連用形,する,シ,シ
た	助動詞,*,*,*,特殊・タ,基本形,た,タ,タ
。	記号,句点,*,*,*,*,。,。,。


Even simpler tokenizer<br>
wakati (分ち書き）returns only surface tokens

In [11]:
for token in t.tokenize(X[0], wakati=True):
    print(token)

父
は
私
が
外国
へ
行く
こと
を
承知
し
た
。


# Identify words to be replaced / ignored

## Data

Our df are 10000 random segments from the SNOW T15 dataset.
df150 are 150 random segments from the SNOW T15 dataset.

In [245]:
df.head()

,#日本語(原文),#やさしい日本語,#英語(原文)
0,父は私が外国へ行くことを承知した。,父は私が外国へ行くことを許した。,my father consented to my going abroad .
1,卑屈な奴。,自分のことをダメだと考える人。,don 't be a dog .
2,それは本当のはずはない。,それは本当のはずはない。,it cannot be true .
3,車がそんなに混んでなければ問題ないでしょう。,車がそんなに混んでなければ問題ないでしょう。,we shouldn 't have any trouble if the traffic ...
4,２時間も待たされた。,２時間も待った。,i was kept waiting for as long as two hours .


X and y are the 0th and 1st columns of df,

In [ ]:
X[0], y[0], len(X), len(y)

In [14]:
node = tagger.parseToNode(text)
while node:
    features = node.feature.split(",")
    original = features[6]
    print('node.surface={},\n node.feature={} => original = {}, \n\n'.format(node.surface,node.feature,original))
    node = node.next

node.surface=,
 node.feature=BOS/EOS,*,*,*,*,*,*,*,*,*,*,*,*,*,*,*,*,*,*,*,*,*,*,*,*,* => original = *, 


node.surface=月曜,
 node.feature=名詞,普通名詞,副詞可能,*,*,*,ゲツヨウ,月曜,月曜,ゲツヨー,月曜,ゲツヨー,漢,*,*,*,*,ゲツヨウ,ゲツヨウ,ゲツヨウ,ゲツヨウ,*,*,"3,0",C2,* => original = ゲツヨウ, 


node.surface=日,
 node.feature=名詞,普通名詞,副詞可能,*,*,*,ヒ,日,日,ヒ,日,ヒ,和,ヒ混合,基本形,*,*,ヒ,ヒ,ヒ,ヒ,*,*,"0,1",C3,* => original = ヒ, 


node.surface=まで,
 node.feature=助詞,副助詞,*,*,*,*,マデ,まで,まで,マデ,まで,マデ,和,*,*,*,*,マデ,マデ,マデ,マデ,*,*,*,"名詞%F2@1,形容詞%F2@1,動詞%F2@1",* => original = マデ, 


node.surface=に,
 node.feature=助詞,格助詞,*,*,*,*,ニ,に,に,ニ,に,ニ,和,*,*,*,*,ニ,ニ,ニ,ニ,*,*,*,名詞%F1,* => original = ニ, 


node.surface=この,
 node.feature=連体詞,*,*,*,*,*,コノ,此の,この,コノ,この,コノ,和,*,*,*,*,コノ,コノ,コノ,コノ,*,*,0,*,* => original = コノ, 


node.surface=仕事,
 node.feature=名詞,普通名詞,サ変可能,*,*,*,シゴト,仕事,仕事,シゴト,仕事,シゴト,和,*,*,*,*,シゴト,シゴト,シゴト,シゴト,*,*,0,C2,* => original = シゴト, 


node.surface=を,
 node.feature=助詞,格助詞,*,*,*,*,ヲ,を,を,オ,を,オ,和,*,*,*,*,ヲ,ヲ,ヲ,ヲ,*,*,*,"動詞%F2@0,名詞%F1,形容詞%F2@-1",* => original = ヲ, 


node.su

## Load word2Vec model

In [6]:
model_path = '/home/andi/code/PlayGround/ShiroYagi/word2vec.gensim.model'
model = Word2Vec.load(model_path)

INFO:gensim.utils:loading Word2Vec object from /home/andi/code/PlayGround/ShiroYagi/word2vec.gensim.model
INFO:gensim.utils:loading wv recursively from /home/andi/code/PlayGround/ShiroYagi/word2vec.gensim.model.wv.* with mmap=None
INFO:gensim.utils:loading syn0 from /home/andi/code/PlayGround/ShiroYagi/word2vec.gensim.model.wv.syn0.npy with mmap=None
INFO:gensim.utils:loading syn1neg from /home/andi/code/PlayGround/ShiroYagi/word2vec.gensim.model.syn1neg.npy with mmap=None
INFO:gensim.utils:setting ignored attribute syn0norm to None
INFO:gensim.utils:setting ignored attribute cum_table to None
INFO:gensim.utils:Word2Vec lifecycle event {'fname': '/home/andi/code/PlayGround/ShiroYagi/word2vec.gensim.model', 'datetime': '2023-02-25T16:39:36.931226', 'gensim': '4.3.0', 'python': '3.8.12 (default, Oct  2 2022, 16:53:09) \n[GCC 11.2.0]', 'platform': 'Linux-5.15.79.1-microsoft-standard-WSL2-x86_64-with-glibc2.35', 'event': 'loaded'}


In [7]:
wv = model.wv

## Generate simp_term list

In [11]:
def clean_data(data):
    """
    Remove English column, rename columns
    """
    data = df.drop(columns=['#英語(原文)'])
    data.rename(columns={"#日本語(原文)": "original", "#やさしい日本語": "simplified"}, inplace=True)
    
    return data

df_clean = clean_data(df)

In [9]:
# Counts all the independent word 自立語
_stopwords = stopwords.words('japanese')

def count_all_words(docs, col='original'):
    all_words = collections.Counter()
    t = MeCab.Tagger("-O wakati")
    for idx, row in docs.iterrows():
        text = row[col]
        node = t.parseToNode(text).next
        while node.next:
            part_of_speech = node.feature.split(',')[0]   
            if part_of_speech in ["助動詞", "助詞", "補助記号"] or node.surface in _stopwords:
                node = node.next
                continue
            all_words[node.surface] += 1
            node = node.next
            
    return all_words

In [12]:
def create_terms(data):
    # corpus of all sentences, where simplication differs from original
    diff_corpus_df = data[data['original'] != data['simplified']]
    # Count all words in the original and simplified column. Ignore stopwords and certain POS
    original_temp_list = count_all_words(diff_corpus_df, 'original')
    diff_temp = count_all_words(diff_corpus_df, 'simplified')
    diff_temp.subtract(original_temp_list)
    diff_temp_df = pd.DataFrame(dict(diff_temp).items(), columns=['word', 'count'])
    diff_temp_df = diff_temp_df[diff_temp_df['count'] >= 0]
    # Make this a list
    term_list = diff_temp_df['word'].to_list()
    
    return term_list

term_list = create_terms(df_clean)
print(len(term_list))

1942


In [13]:
len(term_list)

1942

## Iterate over data and remove terms

In [14]:
# old version that could not handle romaji

def replace_terms(data, term_list=term_list, wv=wv):
    """
    1. Identify every POS in a sentence and if it should be replaced
    2. Use the pre-trained Word2Vec model to get a term from term_list with closest distance to POS
    3. Replace POS in sentence
    4. Add new sentence to dataframe in column "prediction"
    
    input:
    data, np.series
    term_list, list of simplified terms
    wv, word2vec model.wv
    
    output:
    prediction, np.series
    """
    logging.root.setLevel(logging.DEBUG)
    
    start = time.time()
    # Make sure the data is a series, not a df or list
    try:
        assert type(data) == pd.core.series.Series
        logging.info("Data file type OK")
    except:
        print("Data file type is NOT a pd.series")
    
    pos_list = ("名詞", "動詞", "代名詞") # POS (part of speech) that will possibly be removed
    threshold = 0.5 # Threshold of similarity, over which a term will be replaced
    t = MeCab.Tagger()
    numbers_dict = {
        "0":"零",
        "1":"一",
        "2":"二",
        "3":"三",
        "4":"四",
        "5":"五",
        "6":"六",
        "7":"七",
        "8":"八",
        "9":"九"
    }
    counter = collections.Counter()
    prediction = data.copy()
    assert len(prediction) == len(data)     # Make sure prediction and data have the same size
    
    # Iterate over every sentence in the dataset
    for idx, row in data.items():
        row = neologdn.normalize(row)
        logging.debug(f"Currrent sentence: {row}")
        numerals = sum(c.isdigit() for c in row)
        if numerals > 0:
            for i in range(numerals):
                for entry in numbers_dict:
                    row = row.replace(entry, numbers_dict[entry])
        sentence = []
        # Iterate over every word in the sentence
        node = t.parseToNode(row).next
        while node.next:
            word = node.feature.split(',')[8]
            part_of_speech = node.feature.split(',')[0]
            # If POS is not noun, pronoun or verb: add word to list and continue
            if part_of_speech not in pos_list:
                sentence.append(word)
            else:
                # If the term is already in the term list: do not replace, add word to list and continue
                if word in term_list:
                    sentence.append(word)
                else:
                    # Replace word with closest word from term list
                    try:
                        if wv.most_similar(word)[0][1] > threshold:
                            closest_word = wv.most_similar(word)[0][0]
                            sentence.append(closest_word)
                        else:
                            sentence.append(word)
                    except KeyError as e:
                        sentence.append(word)
                        logging.warning(f"{e}. Term will not be replaced.")
            counter[node.surface] += 1
            node = node.next
        logging.debug(sentence)
        prediction[idx] = "".join(sentence)
    
    assert len(data) == len(prediction)  # Make sure prediction and data have the same size
    end = time.time()
    logging.info(end-start)
    return prediction

In [47]:
# new version that can handle romaji, also changed the way it handles numerals

def is_romaji(string):
    """
    returns True if string contains Romaji or a number, False if not.
    """
    alphanum_full = r'[A-z0-9]'
    if re.findall(alphanum_full, string):
        return True
    return False

def replace_terms(data, term_list=term_list, wv=wv):
    """
    1. Identify every POS in a sentence and if it should be replaced
    2. Use the pre-trained Word2Vec model to get a term from term_list with closest distance to POS
    3. Replace POS in sentence
    4. Add new sentence to dataframe in column "prediction"
    
    input:
    data, np.series
    term_list, list of simplified terms
    wv, word2vec model.wv
    
    output:
    prediction, np.series
    """
    logging.root.setLevel(logging.DEBUG)
    
    start = time.time()
    # Make sure the data is a series, not a df or list
    try:
        assert type(data) == pd.core.series.Series
        logging.info("Data file type OK")
    except:
        print("Data file type is NOT a pd.series")
    
    pos_list = ("名詞", "動詞", "代名詞") # POS (part of speech) that will possibly be removed
    threshold = 0.5 # Threshold of similarity, over which a term will be replaced
    t = MeCab.Tagger()
    counter = collections.Counter()
    prediction = data.copy()
    assert len(prediction) == len(data)     # Make sure prediction and data have the same size
    
    # Iterate over every sentence in the dataset
    for idx, row in data.items():
        row = neologdn.normalize(row)
        logging.debug(f"Current sentence: {row}")
        sentence = []
        # Iterate over every word in the sentence
        node = t.parseToNode(row).next
        while node.next:
            logging.debug(f"Current word: {node.feature}")
            if is_romaji(node.surface):
                sentence.append(node.surface)
                node = node.next
            word = node.feature.split(',')[6]
            part_of_speech = node.feature.split(',')[0]
            # If POS is not noun, pronoun or verb: add word to list and continue
            if part_of_speech not in pos_list:
                sentence.append(word)
            else:
                # If the term is already in the term list: do not replace, add word to list and continue
                if word in term_list:
                    sentence.append(word)
                else:
                    # Replace word with closest word from term list
                    try:
                        if wv.most_similar(word)[0][1] > threshold:
                            closest_word = wv.most_similar(word)[0][0]
                            sentence.append(closest_word)
                        else:
                            sentence.append(word)
                    except KeyError as e:
                        sentence.append(word)
                        logging.warning(f"{e}. Term will not be replaced.")
            counter[node.surface] += 1
            node = node.next
        logging.debug(f"Final sentence:{sentence}")
        prediction[idx] = "".join(sentence)
    
    assert len(data) == len(prediction)  # Make sure prediction and data have the same size
    end = time.time()
    logging.info(end-start)
    return prediction

In [ ]:
predictions = replace_terms(X)

INFO:root:Data file type OK
DEBUG:root:Current sentence: 父は私が外国へ行くことを承知した。
DEBUG:root:Current word: 名詞,一般,*,*,*,*,父,チチ,チチ
DEBUG:root:Current word: 助詞,係助詞,*,*,*,*,は,ハ,ワ
DEBUG:root:Current word: 名詞,代名詞,一般,*,*,*,私,ワタシ,ワタシ
DEBUG:root:Current word: 助詞,格助詞,一般,*,*,*,が,ガ,ガ
DEBUG:root:Current word: 名詞,一般,*,*,*,*,外国,ガイコク,ガイコク
DEBUG:root:Current word: 助詞,格助詞,一般,*,*,*,へ,ヘ,エ
DEBUG:root:Current word: 動詞,自立,*,*,五段・カ行促音便,基本形,行く,イク,イク
DEBUG:root:Current word: 名詞,非自立,一般,*,*,*,こと,コト,コト
DEBUG:root:Current word: 助詞,格助詞,一般,*,*,*,を,ヲ,ヲ
DEBUG:root:Current word: 名詞,サ変接続,*,*,*,*,承知,ショウチ,ショーチ
DEBUG:root:Current word: 動詞,自立,*,*,サ変・スル,連用形,する,シ,シ
DEBUG:root:Current word: 助動詞,*,*,*,特殊・タ,基本形,た,タ,タ
DEBUG:root:Current word: 記号,句点,*,*,*,*,。,。,。
DEBUG:root:Final sentence:['父', 'は', '貴方', 'が', '外国', 'へ', '行く', '事', 'を', '弁解', '促す', 'た', '。']
DEBUG:root:Current sentence: 卑屈な奴。
DEBUG:root:Current word: 名詞,形容動詞語幹,*,*,*,*,卑屈,ヒクツ,ヒクツ
DEBUG:root:Current word: 助動詞,*,*,*,特殊・ダ,体言接続,だ,ナ,ナ
DEBUG:root:Current word: 名詞,一般,*,*,*,*,奴,ヤッ

In [34]:
text ="何のことを言っているの??"
t = MeCab.Tagger()

In [ ]:
all_terms = collections.Counter()
node = t.parseToNode(text).next
while node.next:
    part_of_speech = node.feature
    print(node.surface.isalpha(), node.feature, node.feature.split(',')[6)
    all_terms[node.surface] += 1
    node = node.next

In [19]:
predictions.str.find("cd").sort_values(ascending=False)
predictions[847]

NameError: name 'predictions' is not defined

In [96]:


def is_romaji(string):
    """
    returns True if string is Romaji, False if not.
    """
    alphanum_full = r'[A-z0-9]'
    if re.findall(alphanum_full, string):
        return True
    return False

text1 = "らりるれろ"
text2 = "54046"
text3 = "ahsg"

print(text1, is_romaji(text1))
print(text2, is_romaji(text2))
print(text3, is_romaji(text3))

らりるれろ False
54046 True
ahsg True


In [109]:
wv.most_similar("し")

[('図っ', 0.6891764998435974),
 ('行なっ', 0.6623058319091797),
 ('促し', 0.6435315012931824),
 ('行っ', 0.641490638256073),
 ('おこなっ', 0.6279498338699341),
 ('はかっ', 0.6224112510681152),
 ('受け', 0.613266110420227),
 ('努め', 0.6071730256080627),
 ('目指し', 0.6017763614654541),
 ('見送っ', 0.5945292115211487)]